In [3]:
!pip install transformers[torch]


     |████████████████████████████████| 1.3MB 8.9MB/s 
     |████████████████████████████████| 890kB 13.8MB/s 
     |████████████████████████████████| 1.1MB 30.4MB/s 
     |████████████████████████████████| 2.9MB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=700e4502ee7e77b245fa6d1b3fec13206e57b14eaa78c5f15c1273d919c67567
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/ml-practico/AllMoviesDetailsCleaned.csv', delimiter=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from dateutil.parser import parse

df = df[~df.release_date.isna()].copy()
df['release_year'] = df.release_date.apply(lambda x: parse(x).year)
df = df[df.release_year > 1970].copy()
df = df[df.vote_average > 0].copy()
df = df[~df.overview.isna()].copy()

len(df)

93787

In [7]:
df.head(2)

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number,release_year
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,21/10/1988,0,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2,1988
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,16/10/1986,0,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3,1986


In [8]:
train = df[df.release_year <= 2015]
test = df[df.release_year > 2015]

len(test) / len(train), len(test)

(0.06719237158917639, 5905)

In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [21]:
train = train.sample(10000)

In [22]:
train_encodings = tokenizer(train.overview.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test.overview.tolist(), truncation=True, padding=True)

In [23]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = IMDbDataset(train_encodings, (train.vote_average > 7.5).astype(int).tolist())
test_dataset = IMDbDataset(test_encodings, (test.vote_average > 7.5).astype(int).tolist())

In [24]:
from sklearn.metrics import roc_auc_score

def compute_metrics(pred):
    return {'roc': roc_auc_score(pred.label_ids, pred.predictions[:, 1])}


In [30]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
50,0.627036
100,0.388709
150,0.411664
200,0.369001
250,0.385549
300,0.395635
350,0.410291
400,0.377925
450,0.373546
500,0.403151


TrainOutput(global_step=1250, training_loss=0.38342421875)

In [31]:
trainer.evaluate()

{'epoch': 2.0, 'eval_loss': 0.7038650512695312, 'eval_roc': 0.6581229331009297}